In [ ]:
import pandas as pd
import numpy as np
import Functions as fns
import matplotlib.pyplot as plt
import pingouin as pg

In [ ]:
# OPTION 1, import your own data: Load all protein data

df = #INPUT dataframe 
X,y =  # INPUT division into X and y of DataFrame depending on association to test. 
main = #INPUT list of main predictor and confounders

print('Uploaded data contains ' + str(len(df)) + ' subjects.')

In [ ]:
# OPTION 2, use the simulated data

df_all = pd.read_csv('simulated_data.csv',sep=';')
all_prots = ['protein1','protein2','protein3','protein4','protein5']

# Make simulated cohort larger
df_all[[col for col in df_all if col != 'sex']] = df_all[[col for col in df_all if col != 'sex']] + np.random.randn(np.shape(df_all)[0],np.shape(df_all)[1]-1)
for i in range(4):
    new_df = df_all[[col for col in df_all if col != 'sex']] + np.random.randn(np.shape(df_all)[0],np.shape(df_all)[1]-1)
    new_df['sex'] = df_all['sex']
    df_all = pd.concat([df_all,new_df])
df_all = df_all.reset_index(drop=True)

print('Uploaded data contains ' + str(len(df_all)) + ' subjects and ' + str(len(all_prots)) + ' proteins.')

main = ['protein1','age'] # INPUT of form [main predictor, confounders]
X = df_all[all_prots[1:] + main] # define X to predict y
y = df_all['sex'] # Example: predict sex

### Evaluate reference in logistic regression model

In [ ]:
# Perform bootstrapping of the roc auc scores with or without references, as single reference, or svd/mean of several.

kind = 0 # 0 for test with a single reference, 1 for test with svd of several references and 2 for test with mean level of several references
prots = all_prots[1:] # INPUT list of protein(s) to use as reference

X_ref = X[main + prots]
auc = fns.bootstrap_roc_auc(X_ref,y,kind=kind,prots=prots)
print('Mean AUC score: ' + str(np.mean(auc)))

In [ ]:
# Plot results

width = 0.5
plt.bar(1,np.mean(auc),width=width)
plt.axis([0,2,0,1])
#plt.grid(alpha=0.3)

In [ ]:
# Print confidence interval 
print('CI lower AUC: ' + str(sorted(auc)[50]))
print('CI upper AUC: ' + str(sorted(auc)[1949]))

### Compare two references in logistic regression model

In [ ]:
prots1 = ['protein2']#INPUT list of first reference
prots2 = ['protein3','protein4']#INPUT list of second reference

# 0 for test with a single reference, 1 for test with svd of several references and 2 for test with mean level of several references
kind1 = 0# INPUT kind for first reference
kind2 = 1# INPUT kind for second reference

X_use = X[main + prots1 + prots2]
n_iter=2000
roc_auc_diff,auc1,auc2 = fns.test_bootstrap_roc_auc(X_use, y, main=main, prots1 = prots1, kind1 = kind1, prots2 = prots2, kind2 = kind2,n_iter=n_iter)

In [ ]:
# Test significance 
print('Mean AUC difference: ' + str(np.mean(roc_auc_diff)))
impr = [val for val in roc_auc_diff if val > 0]
print('P-value AUC difference: ' + str(1 - len(impr)/n_iter))

### Linear regressions 

In [ ]:
main_pred = 'protein1'# INPUT name of main predictor
outcome = 'age'#INPUT name of outcome variable
confounders = ['sex','protein2']#INPUT list of confounders

betas, pvals = fns.get_linreg(df_all, main_pred=main_pred,outcome=outcome,confounders=confounders)

In [ ]:
print('Betas: \n' + str(betas))

In [ ]:
print('P-values: \n' + str(pvals))

### Partial Correlations

In [ ]:
x = 'protein1'# INPUT name of main predictor
y = 'age'#INPUT name of outcome variable
covars = ['sex','protein2']#INPUT list of covariates

p_corr = pg.partial_corr(data=df_all, x=x, y=y, covar=covars).r['pearson']

In [ ]:
print('Partial correlation: ' + str(p_corr))

### ANCOVA test

In [ ]:
protein_name = 'protein1'#INPUT name of protein
group_name = 'sex'#INPUT column name specifying a group (sex as example here, in manuscript AT(N) group)
covars = ['age','protein2']#INPUT list of covariates

res = pg.ancova(data=df_all, dv = protein_name, covar=covars,between=group_name)['p-unc'][0]

In [ ]:
print('pval: ' + str(res))